In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken


In [2]:
load_dotenv('.env')

True

In [3]:
# Pass the API Key to the OpenAI Client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [32]:
df_subjects = pd.read_csv(".\data\moe_subjects.csv")

In [33]:
len(df_subjects['SCHOOL_NAME'].unique())

337

In [36]:
list_of_schools = df_subjects['SCHOOL_NAME'].unique().tolist()

In [46]:
# Create the dictionary
schools_dict = {
    'Location_and_contact': list_of_schools,
    'Subjects': list_of_schools,
    'Co-curricular_activity_or_CCA': list_of_schools
}

print(schools_dict)

{'Location_and_contact': ['ADMIRALTY PRIMARY SCHOOL', 'ADMIRALTY SECONDARY SCHOOL', 'AHMAD IBRAHIM PRIMARY SCHOOL', 'AHMAD IBRAHIM SECONDARY SCHOOL', 'AI TONG SCHOOL', 'ALEXANDRA PRIMARY SCHOOL', 'ANCHOR GREEN PRIMARY SCHOOL', 'ANDERSON PRIMARY SCHOOL', 'ANDERSON SECONDARY SCHOOL', 'ANG MO KIO PRIMARY SCHOOL', 'ANDERSON SERANGOON JUNIOR COLLEGE', 'ANG MO KIO SECONDARY SCHOOL', 'ANGLICAN HIGH SCHOOL', 'ANGLO-CHINESE JUNIOR COLLEGE', 'ANGLO-CHINESE SCHOOL (BARKER ROAD)', 'ANGLO-CHINESE SCHOOL (INDEPENDENT)', 'ANGLO-CHINESE SCHOOL (JUNIOR)', 'ANGLO-CHINESE SCHOOL (PRIMARY)', 'ANGSANA PRIMARY SCHOOL', 'ASSUMPTION ENGLISH SCHOOL', 'ASSUMPTION PATHWAY SCHOOL', 'BARTLEY SECONDARY SCHOOL', 'BEACON PRIMARY SCHOOL', 'BEATTY SECONDARY SCHOOL', 'BEDOK GREEN PRIMARY SCHOOL', 'BEDOK GREEN SECONDARY SCHOOL', 'BEDOK SOUTH SECONDARY SCHOOL', 'BEDOK VIEW SECONDARY SCHOOL', 'BENDEMEER PRIMARY SCHOOL', 'BENDEMEER SECONDARY SCHOOL', 'BLANGAH RISE PRIMARY SCHOOL', 'BOON LAY GARDEN PRIMARY SCHOOL', 'BOON LAY

In [47]:
def get_embedding(input, model='text-embedding-3-small'):
    response = client.embeddings.create(
        input=input,
        model=model
    )
    return [x.embedding for x in response.data]

In [48]:
# This is the "Updated" helper function for calling LLM
def get_completion(prompt, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=1024, n=1, json_output=False):
    if json_output == True:
      output_json_structure = {"type": "json_object"}
    else:
      output_json_structure = None

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create( #originally was openai.chat.completions
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1,
        response_format=output_json_structure,
    )
    return response.choices[0].message.content

In [49]:
# Note that this function directly take in "messages" as the parameter.
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, top_p=1.0, max_tokens=1024, n=1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1
    )
    return response.choices[0].message.content

In [ ]:
def identify_category_and_school(user_message):
    delimiter = "####"

    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be enclosed in
    the pair of {delimiter}.

    Decide if the query is relevant to any specific courses
    in the Python dictionary below, which each key is a `category`
    and the value is a list of `school_name`.

    If there are any relevant school name(s) found, output the pair(s) of a) `school_name` the relevant schools and b) the associated `category` into a
    list of dictionary object, where each item in the list is a relevant school
    and each school is a dictionary that contains two keys:
    1) category
    2) school_name

    {schools_dict}

    If are no relevant school names found, output the category.

    Ensure your response contains only the list of dictionary objects or an empty list, \
    without any enclosing tags or delimiters.
    """

    messages =  [
        {'role':'system',
         'content': system_message},
        {'role':'user',
         'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    category_and_product_response_str = get_completion_from_messages(messages)
    category_and_product_response_str = category_and_product_response_str.replace("'", "\"")
    category_and_product_response = json.loads(category_and_product_response_str)
    return category_and_product_response

In [81]:
# Testing the function to make sure it works
# This part should not be included as part of the Python script.

user_query = "I am interested in knowing the address of Dunman High and Raffles Institution and what mrt stations are nearby. Also tell me which school offers more subjects."
result = identify_category_and_school(user_query)
print(result) 

[{'category': 'Location_and_contact', 'school_name': 'DUNMAN HIGH SCHOOL'}, {'category': 'Location_and_contact', 'school_name': 'RAFFLES INSTITUTION'}, {'category': 'Subjects', 'school_name': 'DUNMAN HIGH SCHOOL'}, {'category': 'Subjects', 'school_name': 'RAFFLES INSTITUTION'}]


In [65]:
with open('./data/transformed_address.json', 'r') as file:
    json_string = file.read()
    location_dict = json.loads(json_string)

with open('./data/transformed_subjects.json', 'r') as file:
    json_string = file.read()
    subject_dict = json.loads(json_string)

with open('./data/transformed_cca.json', 'r') as file:
    json_string = file.read()
    cca_dict = json.loads(json_string)


In [ ]:
def process_items(items, location_data, subjects_data, cca_data):
    combined_data = {}
    
    for item in items:
        school_name = item['school_name']
        category = item['category']
        
        # Initialize school entry if not exists
        if school_name not in combined_data:
            combined_data[school_name] = {}
        
        # Look up data based on category
        if category == 'Location_and_contact':
            if school_name in location_data:
                # Extract all location columns
                school_location = location_data[school_name]
                combined_data[school_name]['location_and_contact'] = {
                    'address': school_location.get('address', 'N/A'),
                    'postal_code': school_location.get('postal_code', 'N/A'),
                    'telephone': school_location.get('telephone', 'N/A'),
                    'mrt_station': school_location.get('mrt_station', 'N/A')
                }
            else:
                combined_data[school_name]['location_and_contact'] = "Data not found"
                
        elif category == 'Subjects':
            if school_name in subjects_data:
                combined_data[school_name]['subjects'] = subjects_data[school_name]
            else:
                combined_data[school_name]['subjects'] = "Data not found"

        elif category == "Co-curricular_activity_or_CCA":
            if school_name in cca_data:
                combined_data[school_name]['cca'] = cca_data[school_name]
            else:
                combined_data[school_name]['cca'] = "Data not found"
    
    return combined_data



In [83]:
output = process_items(result, location_dict, subject_dict, cca_dict)

output

{'DUNMAN HIGH SCHOOL': {'location_and_contact': {'address': '10 TANJONG RHU ROAD',
   'postal_code': 436895,
   'telephone': '63450533',
   'mrt_station': 'KATONG PARK MRT,ALJUNIED MRT, KALLANG MRT, MOUNTBATTEN MRT,'},
  'subjects': {'name': 'DUNMAN HIGH SCHOOL',
   'subjects_offered': ['Active Citizenry Education',
    'Additional Mathematics',
    'Appreciation of Chinese Culture',
    'Arabic Language as 3rd Lang',
    'Art',
    'Art (Special Programme)',
    'Bahasa Indonesia',
    'Bengali Language',
    'Biology',
    'Chemistry',
    'China Studies in Chinese',
    'China Studies in English',
    'Chinese Language',
    'Chinese Language Syllabus B',
    'Chinese Language and Literature',
    'Computing',
    'Conversational Malay Programme',
    'D1 Biology',
    'D1 Chemistry',
    'D1 Geography',
    'D1 History',
    'D1 Literature in Chinese',
    'D1 Literature in English',
    'D1 Mathematics 2',
    'D1 Physics',
    'D2 Biology',
    'D2 Chemistry',
    'D2 Geography',

In [84]:
def generate_response_based_on_school_details(user_message, school_details):
    delimiter = "####"

    system_message = f"""
    Follow these steps to answer the customer queries.
    The customer query will be delimited with a pair {delimiter}.

    Step 1:{delimiter} If the user is asking about school details, \
    understand the relevant school details in the dict below.
    {school_details}

    Step 2:{delimiter} Use the information about the school details to \
    generate the answer for the customer's query.
    Complete with details such as location, contact information, mrt stations, subjects offered and CCA offered etc.
    Take note especially for the subjects offered. If there are certain subjects present in the {school_details} dict that the customer asked about, you need to tell the customer.
    Take note especially for the co-curricular activities or cca offered. If there are certain cca present in the {school_details} dict that the customer asked about, you need to tell the customer.
    You must only rely on the facts or information in the school details.
    Your response should be as detailed as possible and \
    include information that is useful for customer to better understand the school.

    Step 3:{delimiter}: Answer the customer in a friendly tone.
    Make sure the statements are factually accurate.
    Your response should be comprehensive and informative to help the \
    the customers to make their decision.
    Complete with details such as location, contact information, mrt stations, subjects and CCA offered etc. 
    Take note especially for the subjects offered. If there are certain subjects present in the {school_details} dict that the customer asked about, you need to tell the customer.
    Take note especially for the co-curricular activities or cca offered. If there are certain cca present in the {school_details} dict that the customer asked about, you need to tell the customer.
    Use Neural Linguistic Programming to construct your response.

    Use the following format:
    Step 1:{delimiter} <step 1 reasoning>
    Step 2:{delimiter} <step 2 reasoning>
    Step 3:{delimiter} <step 3 response to customer>

    Make sure to include {delimiter} to separate every step.
    """

    messages =  [
        {'role':'system',
         'content': system_message},
        {'role':'user',
         'content': f"{delimiter}{user_message}{delimiter}"},
    ]

    response_to_customer = get_completion_from_messages(messages)
    response_to_customer = response_to_customer.split("Step 3:####")[1].split("####")[0].strip()
    return response_to_customer

In [85]:
user_query = f"""I am interested in knowing the address of Dunman High and Raffles Institution and what mrt stations are nearby. Also tell me which school offers Tamil language"""

response = generate_response_based_on_school_details(user_query, output)
print(response)

Thank you for your inquiry! Here are the details you requested:

**Dunman High School**
- **Address:** 10 Tanjong Rhu Road, Singapore 436895
- **Nearby MRT Stations:** Katong Park MRT, Aljunied MRT, Kallang MRT, Mountbatten MRT

**Raffles Institution**
- **Address:** 1 Raffles Institution Lane, Singapore 575954
- **Nearby MRT Stations:** Bishan MRT, Marymount MRT

Both Dunman High School and Raffles Institution offer Tamil language as part of their curriculum. If you have any more questions or need further information, feel free to ask!


In [ ]:
def process_user_message(user_input):
    delimiter = "```"

    # Process 1: If schools are found in the user query, look them up in the category and school name dict
    category_and_school_name = identify_category_and_school(user_input)
    print("category and school name : ", category_and_school_name)

    # Process 2: Get the Course Details
    school_details = extract_details(category_and_school_name, location_dict, subject_dict, cca_dict)

    # Process 3: Generate Response based on Course Details
    reply = generate_response_based_on_school_details(user_input, school_details)


    return reply





In [86]:
with open('./data/transformed_address.json', 'r') as file:
    json_string = file.read()
    location_dict = json.loads(json_string)
    print(location_dict)



{'ADMIRALTY PRIMARY SCHOOL': {'name': 'ADMIRALTY PRIMARY SCHOOL', 'address': '11 WOODLANDS CIRCLE', 'postal_code': 738907, 'telephone': '63620598', 'mrt_station': 'Admiralty Station'}, 'ADMIRALTY SECONDARY SCHOOL': {'name': 'ADMIRALTY SECONDARY SCHOOL', 'address': '31 WOODLANDS CRESCENT', 'postal_code': 737916, 'telephone': '63651733', 'mrt_station': 'ADMIRALTY MRT'}, 'AHMAD IBRAHIM PRIMARY SCHOOL': {'name': 'AHMAD IBRAHIM PRIMARY SCHOOL', 'address': '10 YISHUN STREET 11', 'postal_code': 768643, 'telephone': '67592906', 'mrt_station': 'Yishun'}, 'AHMAD IBRAHIM SECONDARY SCHOOL': {'name': 'AHMAD IBRAHIM SECONDARY SCHOOL', 'address': '751 YISHUN AVENUE 7', 'postal_code': 768928, 'telephone': '67585384', 'mrt_station': 'CANBERRA MRT, YISHUN MRT'}, 'AI TONG SCHOOL': {'name': 'AI TONG SCHOOL', 'address': '100 Bright Hill Drive', 'postal_code': 579646, 'telephone': '64547672', 'mrt_station': 'Bishan MRT'}, 'ALEXANDRA PRIMARY SCHOOL': {'name': 'ALEXANDRA PRIMARY SCHOOL', 'address': '2A Prince

In [87]:
# Extract the value of the `dict_of_courses` dictionary
# If you are not sure what the dictionary looks like, you can print it out
list_of_dict = []
for school_name, details_dict in location_dict.items():
    list_of_dict.append(details_dict)

# display the `dict_of_course` as a Pandas DataFrame
df = pd.DataFrame(list_of_dict)
df

,name,address,postal_code,telephone,mrt_station
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,63620598,Admiralty Station
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,63651733,ADMIRALTY MRT
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,67592906,Yishun
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,67585384,"CANBERRA MRT, YISHUN MRT"
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,64547672,Bishan MRT
...,...,...,...,...,...
332,ZHANGDE PRIMARY SCHOOL,51 Jalan Membina,169485,62740357,Tiong Bahru MRT Station
333,ZHENGHUA PRIMARY SCHOOL,9 Fajar Road,679002,67697478,LRT: Fajar Station
334,ZHENGHUA SECONDARY SCHOOL,62 Bukit Panjang Ring Road,679962,67639455,JELAPANG LRT
335,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4,556095,62835413,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT"
